<a href="https://colab.research.google.com/github/Msc-Group-project-Group-2/Msc-Group-Project-Code-1/blob/main/Copy_of_bdv_solar.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
import pandas as pd

In [ ]:
flares = pd.read_csv('/content/drive/MyDrive/_flares.csv')
spots = pd.read_csv('/content/drive/MyDrive/_spots.csv')

flares.head()

<ipython-input-3-5b0ad8495e7a>:2: DtypeWarning: Columns (4) have mixed types. Specify dtype option on import or set low_memory=False.
  spots = pd.read_csv('/content/drive/MyDrive/_spots.csv')


,Date,Time,X-ray Class,Flare Magnitude,NOAA
0,31777810102,441,C,18.0,NaN
1,31777810102,955,C,18.0,NaN
2,31777810103,1025,C,51.0,NaN
3,31777810103,1549,C,22.0,2857.0
4,31777810103,1837,C,22.0,NaN


In [ ]:
spots.head()

,Date,Time,Number of Spots,McIntosh Class,NOAA,Zurich Class,Penumbra Class,Compactness Class
0,11811201,21,32.0,EKI,3478,E,K,I
1,11811201,115,16.0,EHI,3478,E,H,I
2,11811201,210,22.0,EHO,3478,E,H,O
3,11811201,1355,31.0,EHI,3478,E,H,I
4,11811201,1535,17.0,EHI,3478,E,H,I


In [ ]:
# Define the functions
def get_date(value):
    date_data = str(value)[-6:]
    base_year = date_data[:2]
    prefix = '20' if int(base_year) < 18 else '19'
    year = prefix + base_year
    return year + '-' + date_data[2:4] + '-' + date_data[4:6]

def get_time(value):
    value = str(value)
    return (value[:-2] if value[:-2] != "" else "00") + ":" + value[-2:]

from datetime import datetime

def is_valid_datetime(dt_str):
    try:
        # Attempt to parse the value as a valid datetime
        datetime.strptime(dt_str, '%Y-%m-%d %H:%M:%S')
        return True
    except ValueError:
        return False


In [ ]:
# Applying the functions to 'Date' and 'Time' columns
flares['Date'] = flares['Date'].apply(get_date)
flares['Time'] = flares['Time'].apply(get_time)

# Converting 'Time' to string to ensure 4 digits
flares['Time'] = flares['Time'].astype(str).str.zfill(4)

#'Start Time' and 'End Time' inference based on 'Time' (Peak Time)
flares['Start Time'] = pd.to_datetime(flares['Time'], format='%H:%M') - pd.Timedelta(minutes=10)
flares['Start Time'] = flares['Start Time'].dt.time
flares['End Time'] = pd.to_datetime(flares['Time'], format='%H:%M') + pd.Timedelta(minutes=10)
flares['End Time'] = flares['End Time'].dt.time

# 'Start Time' and 'End Time'conversion to string
flares['Start Time'] = flares['Start Time'].astype(str)
flares['End Time'] = flares['End Time'].astype(str)

# Removing ":00" from 'Start Time' and 'End Time'
flares['Start Time'] = flares['Start Time'].str.rstrip(':00')


In [ ]:
flares.head()

,Date,Time,X-ray Class,Flare Magnitude,NOAA,Start Time,End Time
0,1981-01-02,4:41,C,18.0,NaN,04:31,04:51:00
1,1981-01-02,9:55,C,18.0,NaN,09:45,10:05:00
2,1981-01-03,10:25,C,51.0,NaN,10:15,10:35:00
3,1981-01-03,15:49,C,22.0,2857.0,15:39,15:59:00
4,1981-01-03,18:37,C,22.0,NaN,18:27,18:47:00


In [ ]:
# Exclude rows that contain invalid values in the 'Start Time' column
flares = flares[flares['Start Time'].str.contains(':')]
flares = flares[flares['End Time'].str.contains(':')]

# Converting 'Start Time' to datetime
flares['Start Time'] = pd.to_datetime(flares['Date'] + ' ' + flares['Start Time'])

# Concatenating 'Date' and 'Start Time' into 'Start Time' as datetime
# Concatenating 'Date' and 'End Time' into 'End Time' as datetime
flares['End Time'] = pd.to_datetime(flares['Date'] + ' ' + flares['End Time'])

# Calculating 'Flare Duration' in minutes
flares['Flare Duration'] = (flares['End Time'] - flares['Start Time']).dt.total_seconds() / 60

# Converting 'Date' to datetime
flares['Date'] = pd.to_datetime(flares['Date'])

# Extracting the 'Year' from 'Date'
flares['Year'] = flares['Date'].dt.year

# Converting 'NOAA' to type string
flares['NOAA'] = flares['NOAA'].astype(str)

# Drop the 'index' column if it exists
if 'index' in flares.columns:
    flares.drop(columns=['index'], inplace=True)

flares.head()


,Date,Time,X-ray Class,Flare Magnitude,NOAA,Start Time,End Time,Flare Duration,Year
0,1981-01-02,4:41,C,18.0,nan,1981-01-02 04:31:00,1981-01-02 04:51:00,20.0,1981
1,1981-01-02,9:55,C,18.0,nan,1981-01-02 09:45:00,1981-01-02 10:05:00,20.0,1981
2,1981-01-03,10:25,C,51.0,nan,1981-01-03 10:15:00,1981-01-03 10:35:00,20.0,1981
3,1981-01-03,15:49,C,22.0,2857.0,1981-01-03 15:39:00,1981-01-03 15:59:00,20.0,1981
4,1981-01-03,18:37,C,22.0,nan,1981-01-03 18:27:00,1981-01-03 18:47:00,20.0,1981


In [ ]:
flares.to_csv('/content/drive/MyDrive/Mflares.csv', index=False)

In [ ]:
spots.head()

,Date,Time,Number of Spots,McIntosh Class,NOAA,Zurich Class,Penumbra Class,Compactness Class
0,11811201,21,32.0,EKI,3478,E,K,I
1,11811201,115,16.0,EHI,3478,E,H,I
2,11811201,210,22.0,EHO,3478,E,H,O
3,11811201,1355,31.0,EHI,3478,E,H,I
4,11811201,1535,17.0,EHI,3478,E,H,I


In [ ]:
spots['Number of Spots'] = spots['Number of Spots'].apply(int)
spots['Date'] = spots['Date'].apply(get_date)
spots['Time'] = spots['Time'].apply(get_time)
spots['DateTime'] = spots['Date'].str.cat(spots['Time'], sep=' ')
spots['DateTime'] = pd.to_datetime(spots['DateTime'])
spots['Date'] = pd.to_datetime(spots['Date'])
spots['Year'] = spots['Date'].dt.year
spots['Zurich Class'] = spots['Zurich Class'].apply(str)
spots['Penumbra Class'] = spots['Penumbra Class'].apply(str)
spots['Compactness Class'] = spots['Compactness Class'].apply(str)
spots.sort_values(by=['DateTime'], ascending=True, inplace=True)
spots.head()


,Date,Time,Number of Spots,McIntosh Class,NOAA,Zurich Class,Penumbra Class,Compactness Class,DateTime,Year
0,1981-12-01,00:21,32,EKI,3478,E,K,I,1981-12-01 00:21:00,1981
52,1981-12-01,00:21,8,CRO,3481,C,R,O,1981-12-01 00:21:00,1981
46,1981-12-01,00:21,5,BXO,3488,B,X,O,1981-12-01 00:21:00,1981
95,1981-12-01,00:21,6,BXO,3486,B,X,O,1981-12-01 00:21:00,1981
116,1981-12-01,00:21,9,DSO,3485,D,S,O,1981-12-01 00:21:00,1981


In [ ]:
spots.to_csv('/content/drive/MyDrive/Mspots.csv', index=False)